# Beam search
On each step choose k best worlds with lower perplexity - can be k different strategies in one time, that helps not to stuck in local optimum.\
(+) less probability to find local optimum, instead of global optimum, than for greedy approach\
(-) time and resource consuming - you need to score and keep now words for k different strategies

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import numpy as np
import random
import time

In [2]:
df = pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')

In [3]:
df

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


In [4]:
from metric import PerplexityCalculator
scorer = PerplexityCalculator('/kaggle/input/gemma-2/transformers/gemma-2-9b/2')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

This code has an error, so it stopped after 3 rows were scored

In [7]:
tries = 5
beam_size = 10

for idx, text in enumerate(df["text"].to_list()):
    words = text.split()
    print('text', idx)    
    all_candidate_scores = {}
    for attempt in range(tries):
        print(f"Attempt {attempt + 1}/{tries}")
        if attempt == 0:
            # Begin with words having the best perplexity (limited to beam_size)
            initial_perplexities = scorer.get_perplexity(words)
            candidates = [words[i] for i in np.argsort(initial_perplexities)[:beam_size]]
        else:
            # Randomly select beam_size candidates from the unique word set
            candidates = np.random.choice(list(set(words)), beam_size, replace=False)
        
        # Iteratively build candidates by minimizing perplexity
        for step in range(len(words) - 1):
            candidate_scores = {}
            
            for candidate in candidates:
                used_words = candidate.split()
                remaining_words = [word for word in words if word not in used_words]
                
                input_texts = [(candidate + " " + word).strip() for word in remaining_words]
                perplexities = scorer.get_perplexity(input_texts)
                
                # Store perplexity scores for each candidate text
                candidate_scores.update({text: score for text, score in zip(input_texts, perplexities)})
            
            # Select the top beam_size candidates with the lowest perplexities
            candidates = sorted(candidate_scores, key=candidate_scores.get)[:beam_size]
        
        # Store scores from this attempt
        all_candidate_scores.update(candidate_scores)
    
    # Select the best candidate across all attempts
    best_candidate, best_score = min(all_candidate_scores.items(), key=lambda x: x[1])
    df.loc[idx, "beam_greedy_text"] = best_candidate.strip()
    
    # Debugging and validation
    is_length_match = len(best_candidate.strip()) == len(text)
    print(f"Best score: {best_score} Best candidate: {best_candidate} (Length match: {is_length_match})")

text 0
Attempt 1/5
Attempt 2/5
Attempt 3/5
Attempt 4/5
Attempt 5/5
Best score: 515.9932840552193 Best candidate: reindeer mistletoe scrooge elf gingerbread chimney fireplace ornament family advent (Length match: True)
text 1
Attempt 1/5
Attempt 2/5
Attempt 3/5
Attempt 4/5
Attempt 5/5
Best score: 598.5626811957673 Best candidate: reindeer mistletoe elf scrooge gingerbread chimney fireplace ornament advent family night sleep walk laugh jump bake drive give and the (Length match: True)
text 2
Attempt 1/5
Attempt 2/5
Attempt 3/5
Attempt 4/5
Attempt 5/5
Best score: 526.170339675385 Best candidate: nutcracker yuletide grinch gifts holiday decorations ornament stocking holly jingle sleigh carol naughty nice polar beard workshop chimney cheer magi (Length match: True)
text 3
Attempt 1/5
Attempt 2/5
Attempt 3/5
Attempt 4/5
Attempt 5/5


ValueError: min() arg is an empty sequence

Thes code is working good, but i stop this on 5th row after 1st attempt, because it was working 2 hours on attempt

In [23]:
tries = 5
beam_size = 10

for idx, text in enumerate(df["text"].to_list()[3:]):
    words = text.split()
    print('text', idx+3)    
    all_candidate2score = dict()
    for t in range(tries):
        print(f"Tries: {t+1}")
        if t == 0:
            best_primes = scorer.get_perplexity(words)
            candidates = np.array(words)[np.argsort(best_primes)[:beam_size]].tolist()
        else:
            candidates = np.random.choice(list(set(words)), beam_size, replace=False)

        for i in range (len(words) - 1):
            candidate2score = dict()
            for candidate in candidates:
                used_words, remaining_words = candidate.split(), text.split()
                for word in used_words:
                    remaining_words.remove(word)
                input_texts = [(candidate + " " + word).strip() for word in remaining_words]
                perplexities = scorer.get_perplexity(input_texts)
                for input_text, ppl in zip(input_texts, perplexities):
                    candidate2score[input_text] = ppl
            candidate2score = dict(sorted(candidate2score.items(), key=lambda item: item[1]))
            candidates = list(candidate2score.keys())[:beam_size]
            if len(candidate2score) == 0:
                print('candidate2score length 0')
        all_candidate2score.update(candidate2score)
    sorted_by_values = dict(sorted(all_candidate2score.items(), key=lambda item: item[1]))
    best = list(sorted_by_values.items())[0]
    df.loc[idx, "beam_greedy_text"] = best[0].strip()
    
    print(f"{best} (Length match: {is_length_match})")

text 3
Tries: 1
Tries: 2
Tries: 3
Tries: 4
Tries: 5
('ornament yuletide of the holiday decorations and gifts is unwrap stocking holly jingle sleigh nutcracker carol sing cheer cheer grinch naughty nice chimney visit polar beard workshop eat relax magi', 386.4611130887752) (Length match: True)
text 4
Tries: 1
Tries: 2
Tries: 3
Tries: 4
Tries: 5
('eggnog fruitcake chocolate poinsettia peppermint candy wreath snowglobe star angel candle cookie milk and the season of peace joy hope wish dream believe in wonder merry hohoho to you from we have it with wrapping paper bow greeting card toy doll game night puzzle fireplace that not as kaggle workshop', 204.44776193681219) (Length match: True)
text 5
Tries: 1
Tries: 2


KeyboardInterrupt: 

Best text for 5th row after 1st attempt:

In [24]:
sorted_by_values = dict(sorted(all_candidate2score.items(), key=lambda item: item[1]))
best = list(sorted_by_values.items())[0]
df.loc[idx, "beam_greedy_text"] = best[0].strip()

In [25]:
best

('poinsettia yuletide eggnog fruitcake mistletoe nutcracker scrooge grinch snowglobe holly wreath gingerbread cookie peppermint candy sleigh reindeer elf stocking chimney chimney ornament fireplace fireplace ornament angel star candle carol holiday season joy peace cheer merry jingle wish dream believe hope family gifts wrapping paper bow card greeting hohoho laugh sing eat sleep relax unwrap give to and from the and the and the of in is you that it we with as have not of nice naughty night night visit toy doll game puzzle advent chocolate milk cheer jump bake walk drive decorations beard polar workshop workshop wonder magi kaggle',
 82.92766733936132)

In [26]:
df.loc[0, "beam_greedy_text"] = 'reindeer mistletoe scrooge elf gingerbread chimney fireplace ornament family advent'
df.loc[1, "beam_greedy_text"] = 'reindeer mistletoe elf scrooge gingerbread chimney fireplace ornament advent family night sleep walk laugh jump bake drive give and the'
df.loc[2, "beam_greedy_text"] = 'nutcracker yuletide grinch gifts holiday decorations ornament stocking holly jingle sleigh carol naughty nice polar beard workshop chimney cheer magi'
df.loc[3, "beam_greedy_text"] = 'ornament yuletide of the holiday decorations and gifts is unwrap stocking holly jingle sleigh nutcracker carol sing cheer cheer grinch naughty nice chimney visit polar beard workshop eat relax magi'
df.loc[4, "beam_greedy_text"] = 'eggnog fruitcake chocolate poinsettia peppermint candy wreath snowglobe star angel candle cookie milk and the season of peace joy hope wish dream believe in wonder merry hohoho to you from we have it with wrapping paper bow greeting card toy doll game night puzzle fireplace that not as kaggle workshop'
df.loc[5, "beam_greedy_text"] = 'poinsettia yuletide eggnog fruitcake mistletoe nutcracker scrooge grinch snowglobe holly wreath gingerbread cookie peppermint candy sleigh reindeer elf stocking chimney chimney ornament fireplace fireplace ornament angel star candle carol holiday season joy peace cheer merry jingle wish dream believe hope family gifts wrapping paper bow card greeting hohoho laugh sing eat sleep relax unwrap give to and from the and the and the of in is you that it we with as have not of nice naughty night night visit toy doll game puzzle advent chocolate milk cheer jump bake walk drive decorations beard polar workshop workshop wonder magi kaggle'

In [29]:
df['beam_greedy_text'].apply(lambda x: len(x.split()))

0     10
1     20
2     20
3     30
4     50
5    100
Name: beam_greedy_text, dtype: int64

In [28]:
submission = pd.DataFrame(columns=['id', 'text'])

for idx, row in df.iterrows():

    id = row.id
    optimized_text = row.beam_greedy_text

    submission.loc[idx] = {
        'id': id,
        'text': optimized_text
    }

submission.to_csv("submission_beam.csv", index=False)

print('submission created')

submission created
